## Import

In [14]:
import os
import datetime as dt
import pandas as pd
import numpy as np

In [15]:
# Import custom modules - NEED WINDOWS OS
#from CoEQWAL.imports import AuxFunctions as af, cs3, csPlots, cs_util as util, dss3_functions_reference as dss
import AuxFunctions as af, cs3, csPlots, cs_util as util, dss3_functions_reference as dss

OSError: dlopen(../ExcelDSSAdd-inPackage_Feb2016/libraries/64-bit/heclib_x64.dll, 0x0006): tried: '../ExcelDSSAdd-inPackage_Feb2016/libraries/64-bit/heclib_x64.dll' (not a mach-o file), '/System/Volumes/Preboot/Cryptexes/OS../ExcelDSSAdd-inPackage_Feb2016/libraries/64-bit/heclib_x64.dll' (no such file), '/Users/canruso/opt/anaconda3/lib/../ExcelDSSAdd-inPackage_Feb2016/libraries/64-bit/heclib_x64.dll' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Users/canruso/opt/anaconda3/lib/../ExcelDSSAdd-inPackage_Feb2016/libraries/64-bit/heclib_x64.dll' (no such file), '/Users/canruso/opt/anaconda3/lib/../ExcelDSSAdd-inPackage_Feb2016/libraries/64-bit/heclib_x64.dll' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Users/canruso/opt/anaconda3/lib/../ExcelDSSAdd-inPackage_Feb2016/libraries/64-bit/heclib_x64.dll' (no such file), '/Users/canruso/opt/anaconda3/lib/python3.9/lib-dynload/../../../ExcelDSSAdd-inPackage_Feb2016/libraries/64-bit/heclib_x64.dll' (no such file), '/Users/canruso

## Define file names and ranges for DSS and vars

In [16]:
# list of study names file, tab, range, and extension
DssList = r'../dss_may_2024/Exploratory_Results_v03/callite_expl03_run_listing.xlsx'
DssTab = 'callite_expl03_run_listing'
DssMin = 'A1'
DssMax = 'A601'
DssExt = '_DV.dss'
DssOut = 'EDA_06_04_24_dss_names.csv'

# list of relevant variables file, tab, and range (B & C parts)
VarList = r'../Selected Studies and Variables/May27_variables.xlsx'
VarTab = 'May EDA'
VarMin = 'B1'
VarMax = 'C51'
VarOut = 'EDA_vars_06_04_24.csv'

# launch file path to (or CalSim) studies and output csv name
StudyDir = r'../dss_may_2024/Exploratory_Results_v03'
DataDir = r'../data'
DataOut = 'EDA_data_06_04_24.csv'

# WY csv
WYcsv = 'wytypestable.csv'

# time intervals
begYear = 1921
begMonth = 10
begDay = 31
endYear = 2003
endMonth = 9
endDay = 30

## Read names and construct study name

In [ ]:
dsshdr, dssname = af.read_from_excel(DssList, DssTab, DssMin, DssMax, hdr=True)
dss_names = []
for i in range(len(dssname)):
    dss_names.append(dssname[i][0] + DssExt)
dss_names

## Write dss names to file

In [ ]:
dss_df = (pd.DataFrame(dss_names))
dss_df.to_csv(os.path.join(DataDir, DssOut))

## Read variables

In [ ]:
hdr, vars = af.read_from_excel(VarList, VarTab,VarMin,VarMax,hdr=True)
var_df = pd.DataFrame(data=vars, columns=hdr)
var_df

In [ ]:
var_df.to_csv(os.path.join(DataDir,VarOut))

## Create datasets across studies


In [ ]:
def preprocess_data_dss(df, begYear, begMonth, begDay, endYear, endMonth, endDay, addsl=False, addres = True, addpump = True, adddelcvp = True, adddelcvpag = True, addcvpscex = True, addcvpprf = True):
    dvar_list = []
    combined_df = pd.DataFrame()
    TAF_MO_PER_CFS = 60.3707 / 1000
    
    for i, r in df.iterrows():
        dvar_list.append(f'/{r["DSS Part B"]}/{r["DSS Part C"]}/')

    for dss_name in dss_names:

        print(dss_name)

        # Create a blank python "calsim" object
        thiscs3 = cs3.calsim()

        # add start and end dates
        thiscs3.StartDate = dt.datetime(begYear,begMonth,begDay)
        thiscs3.EndDate = dt.datetime(endYear,endMonth,endDay)

        # add path to DSS (CURRENTLY EXPECTING DSS IN THE BASE DIRECTORY, BUT WE SHOULD MAKE IT A PARAMETER)
        DSS_FP = os.path.join(StudyDir, dss_name)
        thiscs3.DV_FP = DSS_FP

        # Retrieve the DSS data variables from the DSS file
        thiscs3.DVdata = cs3.csDVdata(thiscs3)
        thiscs3.DVdata.getDVts(filter=dvar_list)

        df = thiscs3.DVdata.DVtsDF.copy(deep=True)

        # if storage add the 2 variables to create a new one
        # Add S_SLSCVP and S_SLSWP into S_SLTOT

        if addsl:
            df[('CALLITE', 'S_SLTOT', 'STORAGE', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,[('CALLITE', 'S_SLCVP', 'STORAGE', '1MON', '2020D09E', 'PER-AVER', 'TAF'),('CALLITE', 'S_SLSWP', 'STORAGE', '1MON', '2020D09E', 'PER-AVER', 'TAF')]].sum(axis=1)

        if addpump:
            #df[('CALLITE', 'D_JONES_TAF', 'FLOW-DELIVERY', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,('CALLITE', 'D_JONES', 'FLOW-DELIVERY', '1MON', '2020D09E', 'PER-AVER', 'CFS')] * TAF_MO_PER_CFS
            #df[('CALLITE', 'D_BANKS_TAF', 'FLOW-DELIVERY', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,('CALLITE', 'D_BANKS', 'FLOW-DELIVERY', '1MON', '2020D09E', 'PER-AVER', 'CFS')] * TAF_MO_PER_CFS
            #df[('CALLITE', 'D_TOTAL_TAF', 'FLOW-DELIVERY', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,[('CALLITE', 'D_JONES_TAF', 'FLOW-DELIVERY', '1MON', '2020D09E', 'PER-AVER', 'TAF'),('CALLITE', 'D_BANKS_TAF', 'FLOW-DELIVERY', '1MON', '2020D09E', 'PER-AVER', 'TAF')]].sum(axis=1)
            df[('CALLITE', 'D_TOTAL', 'FLOW-DELIVERY', '1MON', '2020D09E', 'PER-AVER', 'CFS')] = df.loc[:,[('CALLITE', 'D_JONES', 'FLOW-DELIVERY', '1MON', '2020D09E', 'PER-AVER', 'CFS'),('CALLITE', 'D_BANKS', 'FLOW-DELIVERY', '1MON', '2020D09E', 'PER-AVER', 'CFS')]].sum(axis=1)

        if addres:
            df[('CALLITE', 'S_RESTOT', 'STORAGE', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,[('CALLITE', 'S_OROVL', 'STORAGE', '1MON', '2020D09E', 'PER-AVER', 'TAF'),('CALLITE', 'S_MELON', 'STORAGE', '1MON', '2020D09E', 'PER-AVER', 'TAF'), ('CALLITE', 'S_SHSTA', 'STORAGE', '1MON', '2020D09E', 'PER-AVER', 'TAF'), ('CALLITE', 'S_MLRTN', 'STORAGE', '1MON', '2020D09E', 'PER-AVER', 'TAF'), ('CALLITE', 'S_FOLSM', 'STORAGE', '1MON', '2020D09E', 'PER-AVER', 'TAF'), ('CALLITE', 'S_TRNTY', 'STORAGE', '1MON', '2020D09E', 'PER-AVER', 'TAF')]].sum(axis=1)
            df[('CALLITE', 'S_RESTOT_NOD', 'STORAGE', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,[('CALLITE', 'S_OROVL', 'STORAGE', '1MON', '2020D09E', 'PER-AVER', 'TAF'),('CALLITE', 'S_SHSTA', 'STORAGE', '1MON', '2020D09E', 'PER-AVER', 'TAF'), ('CALLITE', 'S_TRNTY', 'STORAGE', '1MON', '2020D09E', 'PER-AVER', 'TAF'), ('CALLITE', 'S_FOLSM', 'STORAGE', '1MON', '2020D09E', 'PER-AVER', 'TAF') ]].sum(axis=1)

        if adddelcvp:
            #df[('CALLITE', 'DEL_CVP_TOTAL_N_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,('CALLITE', 'DEL_CVP_TOTAL_N', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')] * TAF_MO_PER_CFS
            #df[('CALLITE', 'DEL_CVP_TOTAL_S_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,('CALLITE', 'DEL_CVP_TOTAL_S', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')] * TAF_MO_PER_CFS
            #df[('CALLITE', 'DEL_CVP_TOTAL_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,[('CALLITE', 'DEL_CVP_TOTAL_N_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF'),('CALLITE', 'DEL_CVP_TOTAL_S_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')]].sum(axis=1)
            df[('CALLITE', 'DEL_CVP_TOTAL', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,[('CALLITE', 'DEL_CVP_TOTAL_N', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS'),('CALLITE', 'DEL_CVP_TOTAL_S', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')]].sum(axis=1)

        if adddelcvpag:
            #df[('CALLITE', 'DEL_CVP_PAG_N_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,('CALLITE', 'DEL_CVP_PAG_N', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')] * TAF_MO_PER_CFS
            #df[('CALLITE', 'DEL_CVP_PAG_S_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,('CALLITE', 'DEL_CVP_PAG_S', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')] * TAF_MO_PER_CFS
            #df[('CALLITE', 'DEL_CVP_PAG_TOTAL_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,[('CALLITE', 'DEL_CVP_PAG_N_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF'),('CALLITE', 'DEL_CVP_PAG_S_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')]].sum(axis=1)
            df[('CALLITE', 'DEL_CVP_PAG_TOTAL', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')] = df.loc[:,[('CALLITE', 'DEL_CVP_PAG_N', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS'),('CALLITE', 'DEL_CVP_PAG_S', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')]].sum(axis=1)

        if addcvpscex:
            #df[('CALLITE', 'DEL_CVP_PSC_N_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,('CALLITE', 'DEL_CVP_PSC_N', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')] * TAF_MO_PER_CFS
            #df[('CALLITE', 'DEL_CVP_PEX_S_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,('CALLITE', 'DEL_CVP_PEX_S', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')] * TAF_MO_PER_CFS
            #df[('CALLITE', 'DEL_CVP_PSCEX_TOTAL_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,[('CALLITE', 'DEL_CVP_PSC_N_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF'),('CALLITE', 'DEL_CVP_PEX_S_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')]].sum(axis=1)
            df[('CALLITE', 'DEL_CVP_PSCEX_TOTAL', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')] = df.loc[:,[('CALLITE', 'DEL_CVP_PSC_N', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS'),('CALLITE', 'DEL_CVP_PEX_S', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')]].sum(axis=1)

        if addcvpprf:
            #df[('CALLITE', 'DEL_CVP_PRF_N_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,('CALLITE', 'DEL_CVP_PRF_N', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')] * TAF_MO_PER_CFS
            #df[('CALLITE', 'DEL_CVP_PRF_S_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,('CALLITE', 'DEL_CVP_PRF_S', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')] * TAF_MO_PER_CFS
            #df[('CALLITE', 'DEL_CVP_PRF_TOTAL_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')] = df.loc[:,[('CALLITE', 'DEL_CVP_PRF_N_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF'),('CALLITE', 'DEL_CVP_PRF_S_TAF', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'TAF')]].sum(axis=1)
            df[('CALLITE', 'DEL_CVP_PRF_TOTAL', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')] = df.loc[:,[('CALLITE', 'DEL_CVP_PRF_N', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS'),('CALLITE', 'DEL_CVP_PRF_S', 'DELIVERY-CVP', '1MON', '2020D09E', 'PER-AVER', 'CFS')]].sum(axis=1)

        new_columns = [(col[0], f'{col[1]}_{dss_name[:-7]}', *col[2:]) if len(col) > 1 else (col[0], '') for col in df.columns]
        df.columns = pd.MultiIndex.from_tuples(new_columns)
        df.columns.names = ['A', 'B', 'C', 'D', 'E', 'F', 'Units']
        combined_df = pd.concat([combined_df, df], axis=1)

    return combined_df

In [ ]:
df = preprocess_data_dss(var_df, begYear, begMonth, begDay, endYear, endMonth, endDay)
df.head(5)

## Write dataframes to file

In [ ]:
df.to_csv(os.path.join(DataDir, DataOut), header=True, index=True)

### Separate Into Dry & Wet Years

In [7]:
df = pd.read_csv(os.path.join(DataDir, DataOut), header=[0, 1, 2, 3, 4, 5, 6], index_col=0, parse_dates=True)
df = df.loc[:, ~df.columns.get_level_values(6).str.contains('CFS.1')]
df.head(2)

A                    CALLITE                                           \
B          C_SJRVER_expl0000 C_WILKNS_expl0000 DEL_CVP_PAG_N_expl0000   
C               FLOW-CHANNEL      FLOW-CHANNEL           DELIVERY-CVP   
D                       1MON              1MON                   1MON   
E                   2020D09E          2020D09E               2020D09E   
F                   PER-AVER          PER-AVER               PER-AVER   
Units                    CFS               CFS                    CFS   
1921-10-31         2505.2790         5427.5645               79.62269   
1921-11-30         1977.8469         4904.4517                0.00000   

A                                                         \
B          DEL_CVP_PAG_S_expl0000 DEL_CVP_PEX_S_expl0000   
C                    DELIVERY-CVP           DELIVERY-CVP   
D                            1MON                   1MON   
E                        2020D09E               2020D09E   
F                        PER-AVER               PER-AVER   
Units                         CFS                    CFS   
1921-10-31              922.12427              978.93085   
1921-11-30              703.33960              336.38205   

A                                                         \
B          DEL_CVP_PMI_N_expl0000 DEL_CVP_PMI_S_expl0000   
C                    DELIVERY-CVP           DELIVERY-CVP   
D                            1MON                   1MON   
E                        2020D09E               2020D09E   
F                        PER-AVER               PER-AVER   
Units                         CFS                    CFS   
1921-10-31              175.15205               177.9746   
1921-11-30              137.90486               239.3647   

A                                                         \
B          DEL_CVP_PRF_N_expl0000 DEL_CVP_PRF_S_expl0000   
C                    DELIVERY-CVP           DELIVERY-CVP   
D                            1MON                   1MON   
E                        2020D09E               2020D09E   
F                        PER-AVER               PER-AVER   
Units                         CFS                    CFS   
1921-10-31              330.14786             1070.54060   
1921-11-30              268.88890              689.73926   

A                                  ...                                       \
B          DEL_CVP_PSC_N_expl0000  ... WYT_SJR__expl0599 WYT_TRIN__expl0599   
C                    DELIVERY-CVP  ...   WATER-YEAR-TYPE    WATER-YEAR-TYPE   
D                            1MON  ...              1MON               1MON   
E                        2020D09E  ...          2020D09E           2020D09E   
F                        PER-AVER  ...          PER-AVER           PER-AVER   
Units                         CFS  ...            NONE.1               NONE   
1921-10-31               776.8217  ...               2.0                3.0   
1921-11-30               990.7904  ...               2.0                3.0   

A                                                               \
B           X2_PRV_expl0599 D_TOTAL_expl0599 S_RESTOT_expl0599   
C          X2-POSITION-PREV    FLOW-DELIVERY           STORAGE   
D                      1MON             1MON              1MON   
E                  2020D09E         2020D09E          2020D09E   
F                  PER-AVER         PER-AVER          PER-AVER   
Units                    KM              CFS               TAF   
1921-10-31         68.20287        9404.3550          9270.104   
1921-11-30         67.41410        7729.3105          9145.287   

A                                                        \
B          S_RESTOT_NOD_expl0599 DEL_CVP_TOTAL_expl0599   
C                        STORAGE           DELIVERY-CVP   
D                           1MON                   1MON   
E                       2020D09E               2020D09E   
F                       PER-AVER               PER-AVER   
Units                        CFS                    TAF   

In [4]:
df_wy = pd.read_csv(os.path.join(DataDir, WYcsv))
df_wy = df_wy[["SACindex_0", "WATERYEAR"]]
df_wy

,SACindex_0,WATERYEAR
0,2,1920
1,2,1921
2,2,1922
3,3,1923
4,5,1924
...,...,...
79,1,1999
80,2,2000
81,4,2001
82,2,2003


In [ ]:
### CHECK MAPPING

In [5]:
wy_value_map = {
    1: "Wet",
    2: "Wet",
    3: "Wet",
    4: "Dry",
    5: "Dry",
}

In [ ]:
def add_water_year_column(df):
    df['Date'] = pd.to_datetime(df.index)
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['WaterYear'] = np.where(df['Month'] >= 10, df['Year'] + 1, df['Year'])
    return df

In [8]:
year_to_category = df_wy.set_index('WATERYEAR')['SACindex_0'].map(wy_value_map).to_dict()
df['Year'] = df.index.year
df['Category'] = df['Year'].map(year_to_category)
df_wet = df[df['Category'] == "Wet"]
df_dry = df[df['Category'] == "Dry"]
df.drop(columns=['Year', 'Category'], inplace=True)

In [12]:
len(df_dry) / 12

30.0

In [13]:
len(df_wet) / 12

52.0

In [ ]:
df_wet.to_csv(os.path.join(DataDir, "wet_" + DataOut), header=True, index=True)

In [ ]:
df_dry.to_csv(os.path.join(DataDir, "dry_" + DataOut), header=True, index=True)

## End of data extaction